**Server code**

In [ ]:
import subprocess
import os

# Define the server details
server_ip = "10.0.62.212"
port = "2022"
username = "urdu_nlp"
password = "nlp#2024"

# Command to run on the server
remote_python_code = """
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Initialize lists to store data
titles = []
urls = []
contents = []

# Function to extract data from a single story URL
def extract_story_data(story_url):
    response = requests.get(story_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract story title
        title = soup.find('h2', class_='txt_blue').get_text(strip=True) if soup.find('h2', class_='txt_blue') else 'No Title'
        # Extract story content
        content_parts = []
        justify_divs = soup.find_all('div', {'align': 'justify'})
        center_divs = soup.find_all('div', {'align': 'center'})

        for div in justify_divs:
            content_parts.append(div.get_text(strip=True))
        for div in center_divs:
            content_parts.append(div.get_text(strip=True))

        content = '\n'.join(content_parts)

        return title, story_url, content
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None, None, None

# Get links for moral stories
moral_stories_links = []
for i in range(1, 2):
    url = f"https://www.urdupoint.com/kids/category/mutafariq-mazameen-page{i}.html"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        stories = soup.find_all('a', class_='sharp_box')
        for story in stories:
            href = story.get('href')
            moral_stories_links.append(href)
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

# Extract data from each story and store it
for story_link in moral_stories_links:
    title, url, content = extract_story_data(story_link)
    if title and url and content:  # Ensure all three are present
        titles.append(title)
        urls.append(url)
        contents.append(content)

# Create a DataFrame
if len(titles) == len(urls) == len(contents):
    df = pd.DataFrame({
        'Title': titles,
        'URL': urls,
        'Content': contents
    })

    # Save DataFrame to CSV file
    csv_file_path = '/content/stories_mazameen.csv'
    df.to_csv(csv_file_path, index=False, encoding='utf-8')

    print(f'Data has been written to {csv_file_path}')
else:
    print("Error: Lists are not of the same length.")
"""

# Write the Python code to a file
with open("remote_code.py", "w") as f:
    f.write(remote_python_code)

# Copy the file to the remote server
command = f"scp -P {port} remote_code.py {username}@{server_ip}:/tmp/remote_code.py"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

# Execute the Python code on the remote server
command = f"ssh -p {port} {username}@{server_ip} 'python /tmp/remote_code.py'"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = process.communicate()

# Display the output
print("Output:\n", stdout.decode())
print("Error (if any):\n", stderr.decode())

# Remove the temporary file
os.remove("remote_code.py")


Output:
 
Error (if any):
 ssh: connect to host 10.0.62.212 port 2022: Connection timed out



**Archives  - on nlp scraping**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the base URL of the blog
base_url = 'https://ncpulblog.blogspot.com/?view=snapshot'

# Send a request to fetch the page content
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Initialize lists to store data
titles = []
urls = []
contents = []

# Find all article links using the class 'post-count-link'
for a_tag in soup.find_all('a', class_='post-count-link', href=True):
    # Extract the article URL
    article_url = a_tag['href']

    # Fetch the article content
    article_response = requests.get(article_url)
    article_soup = BeautifulSoup(article_response.text, 'html.parser')

    # Extract the title of the article
    title_tag = article_soup.find('h3', {'class': 'post-title entry-title'})
    title = title_tag.get_text(strip=True) if title_tag else 'No title found'

    # Extract the content of the article (from <p> tags)
    content_div = article_soup.find('div', {'class': 'post-body entry-content'})
    content = ''
    if content_div:
        for p in content_div.find_all('p'):
            content += p.get_text(strip=True)

    # Append the data to the lists
    titles.append(title)
    urls.append(article_url)
    contents.append(content)

# Step 2: Create a DataFrame from the lists
df = pd.DataFrame({
    'Title': titles,
    'URL': urls,
    'Content': contents
})

# Step 3: Save DataFrame to CSV
df.to_csv('archive.csv', index=False)
print("Scraping completed")


**URDU news - articles**

In [ ]:
!pip install requests beautifulsoup4 pandas

import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL of the website
base_url = 'https://dailyhindustanexpress.com/ur/afkare-jahan/'

# Send a request to fetch the page content
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

# Initialize lists to store data
titles = []
urls = []
contents = []

# Find all article links on the page
for article in soup.find_all('article'):
    # Extract the title and URL
    title_tag = article.find('h3', {'class' : 'jeg_post_title'})  # Adjust the tag as necessary
    if title_tag:
        title = title_tag.get_text(strip=True)
        url = title_tag.find('a')['href']

        # Fetch the article content
        article_response = requests.get(url)
        article_soup = BeautifulSoup(article_response.text, 'html.parser')

        # Extract the content
        content_tag = article_soup.find('div', {'class': 'content-inner'})  # Adjust the class as necessary
        content = content_tag.get_text(strip=True) if content_tag else 'No content found'

        # Append the data to lists
        titles.append(title)
        urls.append(url)
        contents.append(content)

# Create a DataFrame from the lists
df = pd.DataFrame({
    'Title': titles,
    'URL': urls,
    'Content': contents
})

# Save DataFrame to CSV
df.to_csv('articles.csv', index=False)
# df = pd.read_csv("articles.csv")
# df

,Title,URL,Content
0,اسرائیلی وزیراعظم کوامریکی کانگریس سے خطاب کی...,https://dailyhindustanexpress.com/invitation-t...,واشنگٹن: امریکی کانگریس کے ارکان نے اسرائیلی و...
1,ابراہیم رئیسی کی آخری رسوم کی ادائیگی کا عملاً...,https://dailyhindustanexpress.com/ibrahim-rais...,ایک جانب ایران میں فضا سوگوار ہےتو دوسری جانب ...
2,رفیع احمد قدوائی کے ذاتی خطوط نیشنل آرکائیوز ...,https://dailyhindustanexpress.com/rafi-ahmad-q...,نئی دہلی، 8 مئی (یو این آئی) نیشنل آرکائیوز ...
3,مصنوعی ذہانت کا استعمال ایٹم بم کی طرح خطرناک:...,https://dailyhindustanexpress.com/using-artifi...,نئی دہلی: دنیا کے امیر ترین افراد میں سے ایک و...
4,مسکراتے رہیں۔۔۔,https://dailyhindustanexpress.com/keep-smiling/,’انٹرنیشنل ڈے آف ہیپی نیس‘ کے موقع پر فرحت ناز...
5,اسلامو فوبیا پھیلانے کا ذمہ دار سوشل میڈیا: گو...,https://dailyhindustanexpress.com/social-media...,اقوام متحدہ، 16 مارچ (یو این آئی) اقوام متحدہ ...
6,عمران خان اور شاہ محمود کی 12 مقدمات میں ضمانت...,https://dailyhindustanexpress.com/imran-khan-a...,اسلام آباد: راولپنڈی کی انسداد دہشت گردی کی عد...
7,بزرگ اداکارہ تنوجہ اسپتال داخل,https://dailyhindustanexpress.com/veteran-acto...,ممبئی(یواین آئی) معروف بزرگ اداکارہ تنوجہ کو ع...
8,پانچ ریاستوں کے انتخابی نتائج ۔۔۔,https://dailyhindustanexpress.com/election-res...,محمد شرافت علیپانچ ریاستوں کے اسمبلی الیکشن کے...
9,چارریاستوں کا موڈ کیا ہے؟آج پتہ چل جائے گا,https://dailyhindustanexpress.com/what-is-the-...,محمد شرافت علیانتظار کی گھڑی ختم ہوا چاہتی ہے۔...


**Daily jang news**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

# Function to generate date range from start to end
def generate_dates(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    delta = timedelta(days=1)
    while start <= end:
        yield start.strftime("%Y-%m-%d")
        start += delta

# Function to extract article URLs, titles, and content from each page
def scrape_articles(date):
    base_url = f'https://jang.com.pk/category/today-newspaper-archive/{date}'
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    titles = []
    urls = []
    contents = []

    ul_tags = soup.find_all('ul', {"class": "today_editorial"})

    for ul_tag in ul_tags:
        for li_tag in ul_tag.find_all('li'):
            a_tag = li_tag.find('a', href=True)
            if a_tag:
                article_url = a_tag['href']

                # Fetch the article content
                article_response = requests.get(article_url)
                article_soup = BeautifulSoup(article_response.text, 'html.parser')

                # Extract the title
                title_tag = article_soup.find('h1', {'class': 'detail-right-top'})
                title = title_tag.get_text(strip=True) if title_tag else 'No title found'

                # Extract the content
                content_div = article_soup.find('div', {'class': 'detail_view_content'})
                content = ''
                if content_div:
                    content = content_div.get_text(strip=True)

                titles.append(title)
                urls.append(article_url)
                contents.append(content)

    return titles, urls, contents

# Generate a range of dates to scrape (change the start and end date as needed)
start_date = "2008-08-01"
end_date = "2024-09-17"  # You can modify this to limit the range

all_titles = []
all_urls = []
all_contents = []

for date in generate_dates(start_date, end_date):
    titles, urls, contents = scrape_articles(date)
    all_titles.extend(titles)
    all_urls.extend(urls)
    all_contents.extend(contents)

# Create a DataFrame from the lists (limit to the first 20 articles)
df = pd.DataFrame({
    'Title': all_titles,
    'URL': all_urls,
    'Content': all_contents
})

# Save DataFrame to CSV
df.to_csv('jang_archive_articles.csv', index=False)
print("Scraping completed")


**Hindustan express news - nlp_scraping**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the base URL of the website
base_url = 'https://dailyhindustanexpress.com/'

# Initialize lists to store data
titles = []
urls = []
contents = []

# Define a function to fetch articles from a page
def fetch_articles_from_page(page_url):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all articles on the current page
    for article in soup.find_all('article'):
        # Extract the title and URL
        title_tag = article.find('h3', {'class': 'jeg_post_title'})  # Adjust the tag as necessary
        if title_tag:
            title = title_tag.get_text(strip=True)
            url = title_tag.find('a')['href']

            # Fetch the article content
            article_response = requests.get(url)
            article_soup = BeautifulSoup(article_response.text, 'html.parser')

            # Extract the content
            content_tag = article_soup.find('div', {'class': 'content-inner'})  # Adjust the class as necessary
            content = content_tag.get_text(strip=True) if content_tag else 'No content found'

            # Append the data to lists
            titles.append(title)
            urls.append(url)
            contents.append(content)

# Fetch the first page
current_page_url = base_url

# Loop through all pages using the "next" button
while current_page_url:
    # Fetch articles from the current page
    fetch_articles_from_page(current_page_url)

    # Fetch the next page URL
    response = requests.get(current_page_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    next_page_tag = soup.find('a', {'class': 'page_nav next'})  # Adjust the class based on the inspect code
    if next_page_tag:
        next_page_url = next_page_tag['href']
        current_page_url = next_page_url  # Update the URL for the next page
    else:
        current_page_url = None  # Stop the loop if there is no next page

# Create a DataFrame from the lists
df = pd.DataFrame({
    'Title': titles,
    'URL': urls,
    'Content': contents
})

# Save DataFrame to CSV
df.to_csv('hindustanExpress.csv', index=False)
print("Scraping completed")


**URDU moral stories - nlp_scraping**

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Initialize lists to store data
titles = []
urls = []
contents = []

# Function to extract data from a single story URL
def extract_story_data(story_url):
    response = requests.get(story_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract story title
        title = soup.find('h2', class_='txt_blue').get_text(strip=True) if soup.find('h2', class_='txt_blue') else 'No Title'
        # Extract story content
        content_parts = []
        justify_divs = soup.find_all('div', {'align': 'justify'})
        center_divs = soup.find_all('div', {'align': 'center'})

        for div in justify_divs:
            content_parts.append(div.get_text(strip=True))
        for div in center_divs:
            content_parts.append(div.get_text(strip=True))

        content = '\n'.join(content_parts)

        return title, story_url, content
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None, None, None

# Get links for moral stories
moral_stories_links = []
for i in range(1, 133):
    url = f"https://www.urdupoint.com/kids/category/moral-stories-page{i}.html"

    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        stories = soup.find_all('a', class_='sharp_box')
        for story in stories:
            href = story.get('href')
            moral_stories_links.append(href)
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

# Extract data from each story and store it
for story_link in moral_stories_links:
    title, url, content = extract_story_data(story_link)
    if title and url and content:  # Ensure all three are present
        titles.append(title)
        urls.append(url)
        contents.append(content)

# Create a DataFrame
if len(titles) == len(urls) == len(contents):
    df = pd.DataFrame({
        'Title': titles,
        'URL': urls,
        'Content': contents
    })

    # Save DataFrame to CSV file
    csv_file_path = '/content/stories_moral.csv'
    df.to_csv(csv_file_path, index=False, encoding='utf-8')

    print(f'Data has been written to {csv_file_path}')
else:
    print("Error: Lists are not of the same length.")


KeyboardInterrupt: 

**Urdu Stories spider - i200711 - on nlp_scraping**

In [ ]:
!pip install selenium -q
!pip install datasets
!pip install scrapy
!pip install scrapy pandas
!pip install twisted
!pip install w3lib
!pip uninstall -y pyarrow
!pip install pyarrow
!pip install --upgrade pyarrow

import re
from datasets import Dataset
from huggingface_hub import HfApi, HfFolder
import pandas as pd
import os
import pandas as pd
from datasets import Dataset, DatasetDict
from datasets import load_dataset, Dataset
import scrapy
import pandas as pd
from datasets import Dataset, load_dataset
from huggingface_hub import HfApi, HfFolder
import os


%%writefile urdu_story_spider.py

class I200711UrduStoriesSpider(scrapy.Spider):
    name = 'urdu_story'
    start_urls = ['https://www.urduzone.net/']
    count = 2
    max = 226
    url2 = 'https://www.urduzone.net/page/2/?s=+'
    current_page = 2

    # Lists to store data
    ids = []
    urls = []
    contents = []
    record_id = 0  # Start the ID from 0

    def parse(self, response):
        yield scrapy.FormRequest.from_response(
            response,
            formid={'td-header-search': ' '},
            callback=self.parse_next_page
        )

    def parse_next_page(self, response):
        link_container = response.css('div.item-details')
        for next in link_container.css('a::attr(href)').getall():
            yield response.follow(next, callback=self.parse_link)
        yield response.follow(self.url2, callback=self.parse_next_page2)

    def parse_next_page2(self, response):
        link_container = response.css('div.item-details')
        for next in link_container.css('a::attr(href)').getall():
            yield response.follow(next, callback=self.parse_link)

        while self.count < self.max:
            self.count += 1
            self.current_page += 1
            next_page_url = f'https://www.urduzone.net/page/{self.current_page}/?s=+'
            yield response.follow(next_page_url, callback=self.parse_next_page3)

    def parse_next_page3(self, response):
        link_container = response.css('div.item-details')
        for next in link_container.css('a::attr(href)').getall():
            yield response.follow(next, callback=self.parse_link)

    def parse_link(self, response):
        # Capture the URL
        url = response.url

        # Extract the story content
        stories = response.css('*::text').getall()
        urdu_pattern = r'[آ-ے،۔]+'
        urdu_string = ''
        for line in stories:
            if "تین عورتیں تین کہانیاں" in line:
                line = line.replace("تین عورتیں تین کہانیاں", "")
            urdu_words = re.findall(urdu_pattern, line)
            urdu_string += ' '.join(urdu_words) + ' '

        urdu_string += '\n\n'

        # Append data to lists
        self.ids.append(self.record_id)
        self.urls.append(url)
        self.contents.append(urdu_string.strip())
        self.record_id += 1  # Increment the record ID for each story

    def closed(self, reason):
        # Create a DataFrame from the lists
        df = pd.DataFrame({
            'ID': self.ids,
            'URL': self.urls,
            'Content': self.contents
        })

        # Save the DataFrame to a CSV file
        df.to_csv('/content/i200711_urdu_stories.csv', index=False, encoding='utf-8')

        # Optionally, print the DataFrame
        print("DataFrame created:")
        print(df.head())

        # Optionally, check the contents of the CSV file
        with open('/content/i200711_urdu_stories.csv', 'r', encoding='utf-8') as file:
            print(file.read())


!scrapy runspider /content/urdu_story_spider.py

# # Load and display the DataFrame
# df = pd.read_csv('/content/i200711_urdu_stories.csv')
# df

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pya

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/482 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/36.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8307 [00:00<?, ? examples/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/18 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/snehagautam/nlp_scraping/commit/b35798b0ae688595ab85dfe964330ef840e8768c', commit_message='Upload dataset', commit_description='', oid='b35798b0ae688595ab85dfe964330ef840e8768c', pr_url=None, pr_revision=None, pr_num=None)

**Wikipedia - nlp_scraping**

In [ ]:
!pip install selenium -q

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


base_url = 'https://ur.wikipedia.org/wiki/صفحۂ_اول'
try:
    driver = webdriver.Chrome()
except:
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    print("Running in headless mode.")

driver.get(base_url)

time.sleep(3)  # Wait for the page to load

# Extract title and content
title = driver.title
content_elements = driver.find_elements(By.XPATH, '//html/body/div[2]/div/div[3]/main/div[3]/div[3]/div[1]/table[1]/tbody')

# Join all text content
content = "\n".join([element.text for element in content_elements])

# Store the data in a DataFrame
data = {
    'Title': [title],
    'URL': [base_url],
    'Content': [content]
}
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('/content/extracted_urdu_wikipedia_content.csv', index=False, encoding='utf-8')

# df = pd.read_csv('/content/extracted_urdu_wikipedia_content.csv')
# df


Running in headless mode.


,Title,URL,Content
0,ویکیپیڈیا، آزاد دائرۃ المعارف,https://ur.wikipedia.org/wiki/صفحۂ_اول,آزاد دائرۃ المعارف جس میں کوئی بھی ترمیم کر سک...


**Faisaliat - on nlp_scraping**

In [ ]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options


time.sleep(3)  # Wait for the page to load

# Extract title and content
title = driver.title
content_elements = driver.find_elements(By.XPATH, '/html/body/div[1]/div[2]/div[1]/div[2]')

# Join all text content
content = "\n".join([element.text for element in content_elements])

# Store the data in a DataFrame
data = {
    'Title': [title],
    'URL': [base_url],
    'Content': [content]
}
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('faisaliat.csv', index=False, encoding='utf-8')

df = pd.read_csv('/content/faisaliat.csv')
df
